In [2]:
#importing the necessary libraries 

import pandas as pd
from alpha_vantage.timeseries import TimeSeries
from alpha_vantage.alphavantage import AlphaVantage as av  
from pprint import pprint
import json
import urllib.request
from ta import volatility as vol

df = pd.read_csv("C:/Users/user/Downloads/companylist.csv")

data = df['Symbol'][1:101] 

symbols = data.values.tolist() 

symbols

['PIHPP',
 'TURN',
 'FLWS',
 'FCCY',
 'SRCE',
 'VNET',
 'TWOU',
 'TPNL',
 'JOBS',
 'EGHT',
 'AVHI',
 'AAON',
 'ABEO',
 'ABEOW',
 'ABIL',
 'ABMD',
 'AXAS',
 'ACIU',
 'ACIA',
 'ACTG',
 'ACHC',
 'ACAD',
 'ACST',
 'AXDX',
 'XLRN',
 'ANCX',
 'ARAY',
 'ACRX',
 'ACER',
 'ACET',
 'AKAO',
 'ACHV',
 'ACHN',
 'ACIW',
 'ACRS',
 'ACMR',
 'ACNB',
 'ACOR',
 'ATVI',
 'ACXM',
 'ADMS',
 'ADMP',
 'ADAP',
 'ADUS',
 'AEY',
 'IOTS',
 'ADIL',
 'ADILW',
 'ADMA',
 'ADBE',
 'ADOM',
 'ADTN',
 'ADRO',
 'ADES',
 'AEIS',
 'AMD',
 'ADXS',
 'ADXSW',
 'ADVM',
 'DWMC',
 'DWSH',
 'ACT',
 'AEGN',
 'AGLE',
 'AEHR',
 'AMTX',
 'AERI',
 'AVAV',
 'ARPO',
 'AEZS',
 'AEMD',
 'GNMX',
 'AFMD',
 'AGEN',
 'AGRX',
 'AGYS',
 'AGIO',
 'AGMH',
 'AGNC',
 'AGNCB',
 'AGNCN',
 'AGFS',
 'AGFSW',
 'ALRN',
 'AIMT',
 'AIRT',
 'ATSG',
 'AIRG',
 'AMCN',
 'AKAM',
 'AKTX',
 'AKCA',
 'AKBA',
 'AKER',
 'AKRX',
 'AKTS',
 'ALRM',
 'ALSK',
 'ALBO',
 'ABDC']

In [7]:
#defined functions to get requests for the stock symbols for the date '2018-09-27' 

QUERY_URL = "https://www.alphavantage.co/query?function={REQUEST_TYPE}&apikey={KEY}&symbol={SYMBOL}&outputsize={OUTPUTSIZE}"
API_KEY = "7Y9XI9DTQT3PGWBM"


stock_values = [] 


def request(symbol, req_type): 
    with urllib.request.urlopen(QUERY_URL.format(REQUEST_TYPE=req_type, KEY = API_KEY, SYMBOL = symbol, OUTPUTSIZE = 'compact')) as req:
        data = req.read().decode("UTF-8")
    return data


def get_daily_data(symbol):  
    js = json.loads(request(symbol, "TIME_SERIES_DAILY")) 
    jstring = 'Time Series (Daily)'
    
    for entry in js:
        if entry == jstring:  
            i = js[jstring].keys()
            for jkeys in i:
                if jkeys == '2018-09-27':
                    return((jkeys, symbol, 
                            js[jstring][jkeys]['1. open'],
                            js[jstring][jkeys]['2. high'], 
                            js[jstring][jkeys]['3. low'],
                            js[jstring][jkeys]['4. close'],
                            js[jstring][jkeys]['5. volume']))
    return 'api limit'


for symbol in symbols:
    stock_values.append((get_daily_data(symbol)))
    
stock_values


[('2018-09-27', 'PIHPP', '25.1200', '25.1400', '25.1200', '25.1400', '276'),
 ('2018-09-27', 'TURN', '2.1900', '2.2300', '2.1600', '2.2300', '2980'),
 ('2018-09-27', 'FLWS', '11.6000', '11.7750', '11.5500', '11.7000', '242036'),
 ('2018-09-27', 'FCCY', '20.3000', '20.7450', '20.2000', '20.5500', '7878'),
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api limit',
 'api li

In [9]:

stock_df = pd.DataFrame(stock_values)

stock_df = stock_df[[1,3,4,5]][0:4] #symbol, high, low, close

#Using Average True Range as a Volatility Indicator
vol.average_true_range(stock_df[3].astype(float), stock_df[4].astype(float), stock_df[5].astype(float), n=1, fillna=False)

#The stock symbol 'TURN' is the most volatile since the price increases closely by $23 on average
#The stock symbol 'FLWS' is the second volatile stock since the price increases closely by $9.5 on average
#The price of the stock 'FCCY' is the third volatile stock and the price increases by $9 on average
#The stock 'PIHPP' is the least volatile stock because it's price increases only by 0.020 pips


0     0.020
1    22.980
2     9.545
3     9.045
Name: atr, dtype: float64

In [18]:
#another interesting approach that i tried

'''
ts = TimeSeries(key='7Y9XI9DTQT3PGWBM',output_format='pandas',indexing_type='symbol')
#data, metadata = ts.get_daily(symbol='PIHPP',outputsize='compact')
#pprint(data.tail(n=2))


class TimeSeries(av):    
    def get_batch_stock_quotes(self, symbols):
        _FUNCTION_KEY = "BATCH_STOCK_QUOTES"
        return _FUNCTION_KEY, 'Stock Quotes', 'Meta Data'

ts.get_batch_stock_quotes(symbols)
'''